 # Artifical Friendship Construction and Explore the potential social information from the friendship network

In [1]:
import pandas as pd
import feather as ft
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from math import log, e
import time
import pickle

## Important interim results import

In [2]:
""" Mobility dataset import"""
df_wp = ft.read_dataframe('data/weeplace_checkins_without_loc_NA.feather') # it is the dataset without NA location

# # it will be the same if you use the original csv file.
# df_wp= pd.read_csv('data/weeplace_checkins.csv')  # this is original Weeplace dataset without any processing, including some NA location
# df_wp = df_wp.dropna(subset=["placeid",'userid', 'datetime'])

"""Previous results of meetup information import"""
pickle_in = open("meetup_store.pickle", "rb")
meetup_store = pickle.load(pickle_in)
pickle_in.close()

In [192]:
user_meetup=pd.concat(meetup_store)
user_meetup=user_meetup.rename(columns = {'count':'meetup'})
user_meetup = user_meetup.reindex(user_meetup.columns.tolist() + \
                                  ['SN-E ego','LZ-E ego', 'Pi ego', \
                                   'CE alter', 'KL alter', 'Pi alter','CE ego+alter', 'Pi ego+alter', \
                                   'Weight','Impact','Included Rank', \
                                   'CCE ego+alters', 'CCE alters', \
                                   'Pi ego+alters', 'Pi alters'], axis=1)  # version > 0.20.0

In [193]:
user_meetup

,userid_x,userid_y,meetup,percent,SN-E ego,LZ-E ego,Pi ego,CE alter,KL alter,Pi alter,CE ego+alter,Pi ego+alter,Weight,Impact,Included Rank,CCE ego+alters,CCE alters,Pi ego_alters,Pi alters
14,fred-wilson,andrew-parker,10,0.035088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,fred-wilson,bijan-sabet,6,0.021053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,fred-wilson,caroline-mccarthy,5,0.017544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139,fred-wilson,mark-g,5,0.017544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,fred-wilson,eric-spiegelman,4,0.014035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,mark-van-der-poel,simon-colijn,22,0.733333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mark-van-der-poel,rene-sijnke,5,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,mark-van-der-poel,celine-lucas,1,0.033333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mark-van-der-poel,jordi-wiegerink,1,0.033333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Obtain the users' temporal-spatial trajectories

In [3]:
user_list = list(set(df_wp['userid'].tolist()))
N_users = len(user_list)

In [4]:
# # this is dict, dict will more faster!!!
# This produces all users' temporal placeid
user_placeidT = {user_list[i]: df_wp[df_wp['userid'] ==user_list[i]].set_index('datetime').sort_index()[['placeid']] \
                               for i in range(N_users)}

# save pickle file
pickle_out = open("user_placeidT.pickle", "wb")
pickle.dump(user_placeidT, pickle_out)
pickle_out.close()

# # save pickle file
# pickle_out = open("user_meetup_temp.pickle", "wb")
# pickle.dump(user_meetup, pickle_out)
# pickle_out.close()

In [4]:
# read picle file
pickle_in = open("user_placeidT.pickle", "rb")
user_placeidT = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open("user_meetup_temp.pickle", "rb")
user_meetup = pickle.load(pickle_in)
pickle_in.close()

In [8]:
user_meetup

,userid_x,userid_y,meetup,percent,SN-E ego,LZ-E ego,Pi ego,CE alter,KL alter,Pi alter,CE ego+alter,Pi ego+alter,Weight,Impact,Included Rank,CCE ego+alters,CCE alters,Pi alters,Pi ego+alters
14,fred-wilson,andrew-parker,10,0.035088,8.003259,6.059807,0.514013,7.062367,1.002560,0.415810,5.600320,0.557284,0.015719,0.015903,1.0,5.600320,7.062367,0.415810,0.557284
25,fred-wilson,bijan-sabet,6,0.021053,8.003259,6.059807,0.514013,7.821033,1.761225,0.337468,5.762551,0.542123,0.014352,0.014466,2.0,5.495602,7.083546,0.413674,0.567005
34,fred-wilson,caroline-mccarthy,5,0.017544,8.003259,6.059807,0.514013,9.118369,3.058561,0.192075,6.139219,0.506429,0.015133,0.015280,3.0,5.686781,7.529571,0.368031,0.549220
139,fred-wilson,mark-g,5,0.017544,8.003259,6.059807,0.514013,9.583830,3.524023,0.134228,6.368137,0.484389,0.014987,0.015136,4.0,5.847246,7.580211,0.362765,0.534158
69,fred-wilson,eric-spiegelman,4,0.014035,8.003259,6.059807,0.514013,8.769703,2.709896,0.232993,5.718428,0.546259,0.001806,0.001820,5.0,5.738637,7.390469,0.382406,0.544366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,mark-van-der-poel,simon-colijn,22,0.733333,4.585127,3.204491,0.708478,3.664927,0.460436,0.657968,2.944584,0.736171,0.566766,0.601831,1.0,2.944584,3.664927,0.657968,0.736171
2,mark-van-der-poel,rene-sijnke,5,0.166667,4.585127,3.204491,0.708478,6.183090,2.978599,0.343603,3.448809,0.681909,0.320475,0.306636,2.0,3.180504,4.096832,0.608862,0.711059
0,mark-van-der-poel,celine-lucas,1,0.033333,4.585127,3.204491,0.708478,9.031662,5.827171,-0.142373,3.596399,0.665604,0.047478,0.041190,3.0,3.308406,4.247919,0.591276,0.697242
1,mark-van-der-poel,jordi-wiegerink,1,0.033333,4.585127,3.204491,0.708478,10.206009,7.001518,-0.284861,3.936778,0.627259,0.032641,0.025172,4.0,3.514634,4.509532,0.560303,0.674661


## Import useful functions

In [9]:
""" import useful functions"""
from entropy_functions import shannon_entropy, entropy, cross_entropy,LZ_entropy

""" Compute predictability given the lengths of sequences and the LZ-entropy"""
import mpmath

# As required by algorithm, N should be large, we set e as the threshold of N. 
# if it is smaller than threshold, we will just print NA
def getPredictability(N, S, e=100):
    if N >= e:
        f = lambda x: (((1-x)/(N-1)) **(1-x))* x**x - 2**(-S)
        root = mpmath.findroot(f, 1)
        return float(root.real)
    else: 
        return np.nan

In [ ]:
""" compute different entropies, meetup, weights, and predictability"""
f = open('log-run0.txt', 'a', encoding='utf-8')
start_time = time.time()

for ego in user_list[6376:]:        
    ego_temporal_placeid = user_placeidT[ego]
    ego_time = pd.to_datetime(ego_temporal_placeid.index).tolist()
    ego_placeid = ego_temporal_placeid['placeid'].tolist()
    N_uniq_placeid = len(set(ego_placeid))
    length_ego = len(ego_placeid)
    
#     # compute shannon entropy
#     user_meetup.loc[(user_meetup['userid_x']==ego), 'SN-E ego'] = shannon_entropy(ego_placeid)
    
    # compute entropy (it is LZ-entropy without filter)
#     ego_LZ_entropy = entropy(ego_placeid)
    ego_L = entropy(ego_placeid, lambdas=True)

#     user_meetup.loc[(user_meetup['userid_x']==ego),'LZ-E ego'] = ego_LZ_entropy
    
#     # compute predictability (use Fano equation and solve the equation)
#     user_meetup.loc[(user_meetup['userid_x']==ego),'Pi ego'] = getPredictability(length_ego, ego_LZ_entropy, e=2)
    
    # find all the alters for the ego
    # since user_meetup is sorted, so the first one is top frequenct meetup friend
    alters = user_meetup[user_meetup['userid_x']==ego]['userid_y'].tolist()
    
    N_alters = len(alters)
    length_alters = [None] * N_alters

    """For cumulative cross entropy, largest L for all alters, and then get sum"""
    L = [None] * N_alters
    weight = [None] * N_alters
    impact = [None] * N_alters
    
    for j in range(N_alters):
        alter = alters[j]
        # included rank is j+1
#         user_meetup.loc[(user_meetup['userid_x']==ego) & (user_meetup['userid_y']==alter),'Included Rank'] = j+1
        
        alter_temporal_placeid = user_placeidT[alter]
        alter_time = pd.to_datetime(alter_temporal_placeid.index).tolist()
        alter_placeid = alter_temporal_placeid['placeid'].tolist()
        
        """Be careful: W1 in cross_entropy is B in the paper, W2 is cross_entropy is A in the paper """        
        # so we need to get the relative time order of ego in alter (abosulte position of ego+alter)
        # for function cross_entropy, we need to have PTs
        total_time = sorted(ego_time + alter_time)
        PTs = [total_time.index(x) for x in ego_time]
        
        """ function cross_entropy can return L, as defintion of cumulative cross entropy, we need to get max """
        # compute cross entropy with this alter
#         alter_CE = cross_entropy(alter_placeid,ego_placeid, PTs)        
#         user_meetup.loc[(user_meetup['userid_x']==ego) & (user_meetup['userid_y']==alter),'CE alter'] = alter_CE 
#         # compute KL divergence with this alter, cross entropy - entropy
#         user_meetup.loc[(user_meetup['userid_x']==ego) & (user_meetup['userid_y']==alter),'KL alter'] = alter_CE - ego_LZ_entropy 
#         # compute the Pi if only use alter's information to predict ego's future
#         user_meetup.loc[(user_meetup['userid_x']==ego) & (user_meetup['userid_y']==alter),'Pi alter'] = \
#         getPredictability(length_ego, alter_CE, e=2)
        
        L[j] = cross_entropy(alter_placeid,ego_placeid, PTs, lambdas=True)
        # compute the weight: how many times subsequence of A appears in B. If the length  >1, that is to say,
        # at least length-1 subsequence appears in match sequence. 
        weight[j] = np.count_nonzero(np.array(L[j]) - 1)
        # impact is total substring lengths appear in B
        impact[j] = sum(np.array(L[j]) - 1)
        
        length_alters[j] = len(alter_placeid)
        
        # for alters: top above all alters
        alters_L = L[:j+1]
#         alters_Lmax = np.amax(alters_L, axis=0)
        alters_length = length_alters[:j+1]
#         CCE_atlers = (1.0*length_ego/sum(alters_Lmax)) * np.log2(np.mean(alters_length))
#         user_meetup.loc[(user_meetup['userid_x']==ego) & (user_meetup['userid_y']==alter),'CCE alters'] = CCE_atlers 
#         user_meetup.loc[(user_meetup['userid_x']==ego) & (user_meetup['userid_y']==alter),'Pi alters'] = \
#         getPredictability(length_ego, CCE_atlers, e=2)
        
        # for only this alter and ego
        ego_alter_L = [ego_L, L[j]]
        ego_alter_Lmax = np.amax(ego_alter_L, axis=0)
        bi_length = np.array([length_alters[j], length_ego])
        bi_weight = np.array([weight[j], length_ego])
        ave_length = np.mean(bi_length * bi_weight / sum(bi_weight))

        CE_ego_atler = (1.0*length_ego/sum(ego_alter_Lmax)) * np.log2(ave_length)
        user_meetup.loc[(user_meetup['userid_x']==ego) & (user_meetup['userid_y']==alter),'CE ego+alter'] = CE_ego_atler
        user_meetup.loc[(user_meetup['userid_x']==ego) & (user_meetup['userid_y']==alter),'Pi ego+alter'] = \
        getPredictability(length_ego, CE_ego_atler, e=2)
        
        # for ego+alters: top above all alters + ego
        alters_L.append(ego_L)
        ego_alter_CLmax = np.amax(alters_L, axis=0)
        alters_length.append(length_ego)
        ego_alters_weight = weight[:j+1] + [length_ego]
        ave_ego_alters_length = np.mean(np.array(alters_length) * np.array(ego_alters_weight) / sum(ego_alters_weight))
        CCE_ego_atlers = (1.0*length_ego/sum(ego_alter_CLmax)) * np.log2(ave_ego_alters_length)
        user_meetup.loc[(user_meetup['userid_x']==ego) & (user_meetup['userid_y']==alter),'CCE ego+alters'] = CCE_ego_atlers
        user_meetup.loc[(user_meetup['userid_x']==ego) & (user_meetup['userid_y']==alter),'Pi ego+alters'] = \
        getPredictability(length_ego, CCE_ego_atlers, e=2)
        
#     user_meetup.loc[(user_meetup['userid_x']==ego),'Weight'] = np.array(weight)/sum(weight)
#     user_meetup.loc[(user_meetup['userid_x']==ego),'Impact'] = np.array(impact)/sum(impact)
    if (user_list.index(ego) % 100) ==0:
        s = str(user_list.index(ego)) + "--cost--- %s seconds ---" % (time.time() - start_time) \
        + '--CE_ego_atler:'+ str(CE_ego_atler) + '--CCE_ego_alters'+ str(CCE_ego_atlers)
        
        print(s)
        f.write(s)

f.close()
user_meetup.to_csv('user_meetup_update.csv', index=False)

In [32]:
user_list.index(ego)

15754